In [1]:
! pip install pandas numpy faker


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [20]:
# 자산내역 생성

def generate_asset_data(num_users):
    
    # Faker 초기화
    fake = Faker('ko_KR')

    # 사용자 정보 생성
    users = []
    for i in range(num_users):
        user = {
            'name': fake.name(),
            'age': np.random.randint(20, 60),
            'gender': np.random.choice(['남자', '여자']),
            'job': fake.job(),
            'email': fake.email()
        }
        users.append(user)

    # 자산 항목 리스트
    asset_types = ['정기예금', '적금', '주식', '펀드', '보험', '연금', '퇴직연금', '대출']  # '채권' 

    # 데이터프레임 초기화
    asset_data = {
        'User': [],
        'Type': [],
        'Amount': []
    }

    # 자산 내역 생성
    for user in users:
        num_assets = np.random.randint(0, 10)  # 각 사용자별 자산 항목 수
        selected_assets = np.random.choice(asset_types, num_assets, replace=False)  # 랜덤으로 자산 항목 선택

        for asset in selected_assets:
            asset_data['User'].append(user['name'])
            asset_data['Type'].append(asset)

            if asset == '대출':
                amount = -np.random.randint(1000000, 500000000)  # 대출 금액은 -로 표기
            else:
                amount = np.random.randint(100000, 50000000)  # 기타 자산 항목 금액

            asset_data['Amount'].append(amount)

    # 자산 데이터프레임 생성
    asset_df = pd.DataFrame(asset_data)

    # 사용자 데이터프레임 생성
    user_df = pd.DataFrame(users)

    # 출력해서 확인
    print(asset_df)
    print(user_df)

    # 엑셀 파일로 저장
    file_name = './asset_data.xlsx'
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        user_df.to_excel(writer, sheet_name='사용자정보', index=False)
        asset_df.to_excel(writer, sheet_name='자산내역', index=False)

# 함수 호출 예시
generate_asset_data(10)  # 10명 데이터 생성

   User Type    Amount
0   최동현   보험  38651626
1   최동현   연금  24403115
2   최동현   대출 -40884022
3   최동현   적금  37441415
4   최동현   채권  46784956
..  ...  ...       ...
70  전병철   연금  40740279
71  전병철   적금  35572469
72  전병철   보험  43938417
73  전병철   채권  48597859
74  전병철   펀드  18842094

[75 rows x 3 columns]


In [16]:
fake = Faker()

def generate_expenses_for_period(num_users, start_date, end_date, output_file="expenses.csv"):

    # 월고정비 항목 및 비용 범위
    monthly_fixed_costs = {
        "월세": (450000, 550000), "관리비": (80000, 120000),"전기요금": (30000, 70000),"수도요금": (20000, 40000),
        "가스요금": (20000, 40000),"인터넷비": (25000, 35000),"통신비": (40000, 60000),"보험료": (80000, 120000),
        "교통비": (40000, 60000),"자동차할부금": (250000, 350000),"학원비": (150000, 250000),"넷플릭스구독": (12000, 14000),
        "음악구독": (9000, 11000),"대출상환": (450000, 550000),"미용실": (40000, 60000),"적금": (180000, 220000)
    }

    # 연고정비 항목 및 비용 범위
    annual_fixed_costs = {
        "자동차보험료": (500000, 1000000),"헬스장": (450000, 550000),"카드연회비": [5000, 10000, 20000, 50000]
    }

    # 상시비용 (거의매일)항목 및 비용 범위 설정
    daily_expenses_high_freq = {
        "식비": (6000, 20000), "음료": (1000, 10000)
    }

    # 상시비용 (한달 2~5회)항목 및 비용 범위 설정
    monthly_expenses_mid_freq = {
        "주유비": (30000, 100000), "택시비": (3800, 50000),"택배비": (3000, 5000), "도서구매": (3000, 30000),
        "영화": [12000, 15000], "생활용품구매": (10000, 50000)
    }

    # 상시비용 (1년 2~5회)항목 및 비용 범위 설정
    annual_expenses_low_freq = {
        "교육비": (80000, 120000), "병원비": (30000, 70000), "약값": (10000, 30000), "의류비": (20000, 300000),
        "여행비": (500000, 1500000), "가구구매": (300000, 700000), "공연관람": (50000, 100000), "자동차수리": (200000, 400000),
        "화장품구매": (20000, 80000), "건강식품구매": (10000, 50000), "전자제품구매": (500000, 1500000)
    }


    def generate_random_expense(expense_dict):
        """ 주어진 비용 딕셔너리에서 랜덤한 비용 항목과 금액을 반환 """
        key = random.choice(list(expense_dict.keys()))
        value = expense_dict[key]
        if isinstance(value, tuple):
            return key, random.randint(*value)
        elif isinstance(value, list):
            return key, random.choice(value)
        else:
            return key, value

    expenses = []
    for i in range(num_users):
        user_id = f'user_{i+1}'
        current_date = start_date

        # 연간 고정비용 추가
        for expense in annual_fixed_costs.keys():
            expense_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
            amount = annual_fixed_costs[expense]
            if isinstance(amount, tuple):
                amount = random.randint(*amount)
            elif isinstance(amount, list):
                amount = random.choice(amount)
            expenses.append((user_id, expense_date, expense, amount))

        # 지정된 기간 동안 소비 내역 생성
        while current_date <= end_date:
            # 월별 고정비용 추가 (매월 1일)
            if current_date.day == 1:
                for expense, amount_range in monthly_fixed_costs.items():
                    amount = random.randint(*amount_range)
                    expenses.append((user_id, current_date, expense, amount))

            # 빈도 높은 상시비용 추가 (일별로 1-3회)
            for _ in range(random.randint(1, 3)):
                expense, amount = generate_random_expense(daily_expenses_high_freq)
                expenses.append((user_id, current_date, expense, amount))

            # 빈도 중간 상시비용 추가 (한달에 3회 정도)
            if random.randint(1, 10) <= 3:
                for _ in range(random.randint(1, 3)):
                    expense, amount = generate_random_expense(monthly_expenses_mid_freq)
                    expenses.append((user_id, current_date, expense, amount))

            # 빈도 낮은 상시비용 추가 (한 해에 5회 정도)
            if random.randint(1, 365) <= 5:
                for _ in range(random.randint(1, 3)):
                    expense, amount = generate_random_expense(annual_expenses_low_freq)
                    expenses.append((user_id, current_date, expense, amount))

            # 날짜를 하루 증가
            current_date += timedelta(days=1)

    # 데이터프레임 생성
    df = pd.DataFrame(expenses, columns=["user_id", "date", "expense", "amount"])

    # CSV 파일로 저장
    df.to_csv(f'.\data\{output_file}', index=False, encoding='cp949')
    return df

# 생성 유저 수, 시작-끝 일정 설정
num_users = 5
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# 함수 호출
expense_df = generate_expenses_for_period(num_users, start_date, end_date)
print(expense_df.head())  # 데이터프레임의 첫 몇 줄을 출력

# 데이터프레임을 CSV 파일로 저장하는 함수 호출
generate_expenses_for_period(num_users, start_date, end_date, output_file="expenses.csv")


  user_id       date expense  amount
0  user_1 2023-12-27  자동차보험료  735313
1  user_1 2023-07-04     헬스장  510698
2  user_1 2023-11-02   카드연회비   20000
3  user_1 2023-01-01      월세  520031
4  user_1 2023-01-01     관리비   97485


,user_id,date,expense,amount
0,user_1,2023-04-01,자동차보험료,845471
1,user_1,2023-02-19,헬스장,543556
2,user_1,2023-07-16,카드연회비,10000
3,user_1,2023-01-01,월세,508227
4,user_1,2023-01-01,관리비,110711
...,...,...,...,...
5774,user_5,2023-12-30,음료,7619
5775,user_5,2023-12-30,음료,1473
5776,user_5,2023-12-31,식비,17586
5777,user_5,2023-12-31,식비,11744


In [18]:
# 입출금 내역 생성

def generate_bank_transactions(num_users, start_date, end_date, output_file="bank_transactions.csv"):

    # 입출금 항목 리스트
    transaction_types = ['입금', '출금']
    
    # 사용자 번호 리스트 생성
    user_ids = [f"user_{i+1}" for i in range(num_users)]
    
    # 데이터프레임 초기화
    transaction_data = {
        'User_ID': [],
        'Date': [],
        'Transaction_Type': [],
        'Amount': []
    }
    
    # 입출금 내역 생성
    for user_id in user_ids:
        current_date = start_date
        while current_date <= end_date:
            num_transactions = np.random.randint(1, 5)  # 하루에 1~5건의 거래 발생
            for _ in range(num_transactions):
                transaction_type = random.choice(transaction_types)
                if transaction_type == '입금':
                    amount = np.random.randint(1000, 1000000)  # 입금 금액 범위
                else:
                    amount = -np.random.randint(1000, 1000000)  # 출금 금액 범위
                    
                transaction_data['User_ID'].append(user_id)
                transaction_data['Date'].append(current_date)
                transaction_data['Transaction_Type'].append(transaction_type)
                transaction_data['Amount'].append(amount)
                
            current_date += timedelta(days=1)
    
    # 데이터프레임 생성
    transaction_df = pd.DataFrame(transaction_data)
    
    # CSV 파일로 저장
    transaction_df.to_csv(f'.\data\{output_file}', index=False, encoding='cp949')
    
    return transaction_df

# 사용 예시
num_users = 5
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# 함수 호출
transaction_df = generate_bank_transactions(num_users, start_date, end_date)
print(transaction_df.head())  # 데이터프레임의 첫 몇 줄을 출력

  User_ID       Date Transaction_Type  Amount
0  user_1 2023-01-01               입금  881112
1  user_1 2023-01-01               입금  178748
2  user_1 2023-01-01               출금 -618298
3  user_1 2023-01-01               입금  586849
4  user_1 2023-01-02               입금  524350
